**Objective:** To build a model capable of predicting future values using implementing LSTM and other Time Series Techniques. The main goal is to deploy a model capable of predicting the fututre energy values given date and time.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

def load_data():
    df = pd.read_csv('/content/PJMW_hourly.csv', parse_dates=['Datetime'], index_col='Datetime')
    return df

def preprocess_data(df, look_back=24):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df.values)

    X, y = [], []
    for i in range(len(scaled_data)-look_back-1):
        X.append(scaled_data[i:(i+look_back), 0])
        y.append(scaled_data[i + look_back, 0])

    X = np.array(X)
    y = np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    return X, y, scaler

def build_model(input_shape):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model


In [ ]:

# Load and preprocess
df = load_data()
X, y, scaler = preprocess_data(df)

# Build and train
model = build_model((X.shape[1], 1))
model.fit(X, y, epochs=50, batch_size=32, validation_split=0.2)

# Save artifacts
save_model(model, 'lstm_model.h5')
np.save('scaler_params.npy', scaler.get_params())

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3580/3580 ━━━━━━━━━━━━━━━━━━━━ 89s 24ms/step - loss: 0.0084 - val_loss: 5.1325e-04
Epoch 2/50
3580/3580 ━━━━━━━━━━━━━━━━━━━━ 84s 24ms/step - loss: 8.4759e-04 - val_loss: 3.4119e-04
Epoch 3/50
3580/3580 ━━━━━━━━━━━━━━━━━━━━ 146s 25ms/step - loss: 4.7331e-04 - val_loss: 3.5334e-04
Epoch 4/50
3580/3580 ━━━━━━━━━━━━━━━━━━━━ 83s 23ms/step - loss: 4.3525e-04 - val_loss: 3.4444e-04
Epoch 5/50
3580/3580 ━━━━━━━━━━━━━━━━━━━━ 148s 25ms/step - loss: 4.0906e-04 - val_loss: 3.2465e-04
Epoch 6/50
3580/3580 ━━━━━━━━━━━━━━━━━━━━ 141s 25ms/step - loss: 3.8349e-04 - val_loss: 3.1136e-04
Epoch 7/50
3580/3580 ━━━━━━━━━━━━━━━━━━━━ 142s 25ms/step - loss: 3.7712e-04 - val_loss: 3.4281e-04
Epoch 8/50
3580/3580 ━━━━━━━━━━━━━━━━━━━━ 143s 25ms/step - loss: 3.5675e-04 - val_loss: 3.2053e-04
Epoch 9/50
3580/3580 ━━━━━━━━━━━━━━━━━━━━ 141s 25ms/step - loss: 3.3460e-04 - val_loss: 2.8501e-04
Epoch 10/50
3580/3580 ━━━━━━━━━━━━━━━━━━━━ 138s 24ms/step - loss: 3.4331e-04 - val_loss: 2.7241e-04
Epoch 11/50
3580/3580 ━━━━━

In [ ]:
len(X)

143181

In [ ]:
X_train

NameError: name 'X_train' is not defined

In [ ]:
X[1]

array([[0.48885473],
       [0.48292522],
       [0.47985066],
       [0.48786648],
       [0.50938838],
       [0.54968705],
       [0.58603272],
       [0.6011859 ],
       [0.60744482],
       [0.60404085],
       [0.59273087],
       [0.57944438],
       [0.56275393],
       [0.54891841],
       [0.54760075],
       [0.56890304],
       [0.60832327],
       [0.60239376],
       [0.58032283],
       [0.55561656],
       [0.53486329],
       [0.51125508],
       [0.48797628],
       [0.53288679]])

In [ ]:
import joblib
joblib.dump(scaler,"scaler.pkl")

['scaler.pkl']

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, Input
from tensorflow.keras.callbacks import EarlyStopping
import joblib

# --- Load your dataset ---
df = pd.read_csv(r"/content/PJMW_hourly.csv")  # Replace with your actual file
df.columns = df.columns.str.strip()

# Convert 'Datetime' to datetime type
df['Datetime'] = pd.to_datetime(df['Datetime'], errors='coerce')
df.dropna(subset=['Datetime'], inplace=True)

# Feature engineering
df['hour'] = df['Datetime'].dt.hour
df['day'] = df['Datetime'].dt.day
df['weekday'] = df['Datetime'].dt.weekday
df['month'] = df['Datetime'].dt.month

# Select features and target
features = ['hour', 'day', 'weekday', 'month']
target = 'PJMW_MW'
df = df.dropna(subset=[target])

# --- Scale the data ---
scaler = MinMaxScaler()
data = df[features + [target]]
scaled = scaler.fit_transform(data)

# --- Create sequences ---
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length, :-1])  # features
        y.append(data[i+seq_length, -1])     # target
    return np.array(X), np.array(y)

seq_len = 24
X, y = create_sequences(scaled, seq_len)

# --- Train-test split ---
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# --- Build and train model ---
model = Sequential([
    Input(shape=(X_train.shape[1], X_train.shape[2])),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.3),
    LSTM(32),
    Dropout(0.2),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

early_stop = EarlyStopping(patience=15, restore_best_weights=True)
model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=64,
    validation_data=(X_test, y_test),
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/100
1790/1790 ━━━━━━━━━━━━━━━━━━━━ 84s 44ms/step - loss: 0.0127 - val_loss: 0.0063
Epoch 2/100
1790/1790 ━━━━━━━━━━━━━━━━━━━━ 79s 44ms/step - loss: 0.0054 - val_loss: 0.0052
Epoch 3/100
1790/1790 ━━━━━━━━━━━━━━━━━━━━ 86s 46ms/step - loss: 0.0039 - val_loss: 0.0051
Epoch 4/100
1790/1790 ━━━━━━━━━━━━━━━━━━━━ 141s 46ms/step - loss: 0.0036 - val_loss: 0.0052
Epoch 5/100
1790/1790 ━━━━━━━━━━━━━━━━━━━━ 77s 43ms/step - loss: 0.0035 - val_loss: 0.0049
Epoch 6/100
1790/1790 ━━━━━━━━━━━━━━━━━━━━ 82s 46ms/step - loss: 0.0035 - val_loss: 0.0050
Epoch 7/100
1790/1790 ━━━━━━━━━━━━━━━━━━━━ 146s 48ms/step - loss: 0.0034 - val_loss: 0.0051
Epoch 8/100
1790/1790 ━━━━━━━━━━━━━━━━━━━━ 146s 50ms/step - loss: 0.0034 - val_loss: 0.0049
Epoch 9/100
1790/1790 ━━━━━━━━━━━━━━━━━━━━ 140s 49ms/step - loss: 0.0033 - val_loss: 0.0051
Epoch 10/100
1790/1790 ━━━━━━━━━━━━━━━━━━━━ 137s 46ms/step - loss: 0.0033 - val_loss: 0.0049
Epoch 11/100
1790/1790 ━━━━━━━━━━━━━━━━━━━━ 83s 46ms/step - loss: 0.0033 - val_loss:

In [ ]:
# Save the trained model ---
model.save("power_model.h5")
print("Model saved to power_model.h5")

#Save the fitted scaler ---
joblib.dump(scaler, "scaler.pkl")
print("Scaler saved to scaler.pkl")

model = LogisticRegression()

model.fit(x_train, y_train)

y_pred = model.predict(x_test)  ---> y_test
accuracy_score(y_test, y_pred)
89%

In [ ]:
# training (colab)
joblib.dump(model,"lr_model.joblib")

In [ ]:
# app
model = joblib.load("lr_model.joblib")
prediction = model.predict(user_input[10, 20, 30, 40])
prediction
